## Question Embedding extraction through VQA without images

In this notebook we're going to use a Recursive Neural Network for generating Question embeddings based on the question format and it's answer.

The goal of this is to extract array representation for the questions where similar questions are close.

### Load the data

In [1]:
import findspark
findspark.init(findspark.find())

In [2]:
import tensorflow as tf
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

True

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from nltk.tokenize import word_tokenize
from collections import Counter
from keras.preprocessing.sequence import *
from keras.models import *
from keras.layers import *
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

sns.set(style="ticks")

spark = SparkSession \
    .builder \
    .appName("QuestionRephrasing-AutoEncoder") \
    .config("spark.executor.memory", "14G")\
    .config("spark.driver.memory", "14G")\
    .config("spark.driver.maxResultSize", "14G")\
    .getOrCreate()

spark.sparkContext.setCheckpointDir('data/checkpoints')
questions = spark.read.parquet("data/processed/union/*")
questions.printSchema()
questions.count()

Using TensorFlow backend.


root
 |-- question: string (nullable = true)
 |-- answer: string (nullable = true)
 |-- image_id: string (nullable = true)
 |-- tokenized_question: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- question_len: double (nullable = true)
 |-- question_word_len: double (nullable = true)
 |-- first_word: string (nullable = true)



1895874

### Question filtering based on answer frequencies

we're dealing with a dataset that has a wide variety of answers. So for this we'll consider only the answers that appear often in order to reduce the classes predicted.

In [4]:
answers_counts = questions.select('answer')\
    .toPandas()['answer']\
    .value_counts()
answers_counts = pd.DataFrame(list(zip(answers_counts.keys(), answers_counts.values)))
most_common_answers = answers_counts[answers_counts[1] > 600][0]
f"We're going to target only {most_common_answers.count()} answers"

"We're going to target only 332 answers"

In [5]:
most_common_answers_df = spark.createDataFrame([[x] for x in most_common_answers.tolist()], ["answer"])

In [6]:
print(f"Original question size: {questions.count()}, after filtering {questions.join(most_common_answers_df, 'answer').count()}")
questions = questions.join(most_common_answers_df, 'answer')

Original question size: 1895874, after filtering 1038866


In [7]:
max_word_len = int(questions.agg({"question_word_len": "max"}).collect()[0]["max(question_word_len)"])

f"Mmaximum number of word in a question is {max_word_len}."

'Mmaximum number of word in a question is 28.'

### Build the vocabulary

In [8]:
# Tokens vocabulary and mappers
tokens = questions.select('tokenized_question')\
    .rdd\
    .flatMap(lambda x: x['tokenized_question'])\
    .collect()

word_mapping = {}
word_mapping_reversed = {}
word_counter = Counter(tokens)
for idx, value in enumerate(word_counter):
    word_mapping[value] = idx
    word_mapping_reversed[idx] = value
    
f"Word mapping example for 'is': {word_mapping['is']}."

"Word mapping example for 'is': 15."

### Extract the word representation of input using the mapping above

In [9]:
extract_word_embeddings = F.udf(lambda tokenized_question: [word_mapping[word] + 1 for word in tokenized_question], ArrayType(IntegerType()))

questions = questions.withColumn('question_word_embeddings', extract_word_embeddings(F.col('tokenized_question')))
questions.head(1)

[Row(answer='plane', question="what's in the sky?", image_id='21926', tokenized_question=['what', "'s", 'in', 'the', 'sky', '?'], question_len=18.0, question_word_len=6.0, first_word='what', question_word_embeddings=[1, 2, 3, 4, 5, 6])]

In [10]:
word_embeddings = questions.select('question_word_embeddings')\
    .rdd\
    .map(lambda x: x['question_word_embeddings'])\
    .collect()
word_embeddings = pad_sequences(word_embeddings, maxlen=max_word_len, dtype='int32', padding='post', truncating='pre', value=0.0)
word_embeddings[:1]

array([[1, 2, 3, 4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]])

### Transform the answers targets to one-hot representation

In [11]:
answer_tokens = questions.select('answer')\
    .rdd\
    .map(lambda x: x['answer'])\
    .collect()

nr_answers = len(set(answer_tokens))

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(answer_tokens)
onehot_encoder = OneHotEncoder(sparse=True)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)

y = onehot_encoder.fit_transform(integer_encoded)
y[:1]

<1x332 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

### Model for extracting question embeddings

**Input:** question embedded (each token represents a number according to the above mapping) as sequences and padded

**Output:** answers as one-hot

For extracting the question embeddings, we'll train the model and then we'll drop the last layer.

In [12]:
from keras.layers import Dropout


model = Sequential()
model.add(Embedding(len(word_mapping) + 1, 50, input_length=max_word_len, mask_zero=True))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100, activation='relu', dropout=0.1, recurrent_dropout=0.1), input_shape=(max_word_len, 50)))
model.add(Dropout(0.3))
model.add(Dense(50))
model.add(Dense(nr_answers, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 50)            840850    
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 50)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               120800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                10050     
_________________________________________________________________
dense_2 (Dense)              (None, 332)               16932     
Total params: 988,632
Trai

### Compute answers class weights

Since there are lots of answers the dataset is very imbalanced. Thus for optimizing the training we're providing class weights so the model can adjust based on how many times the answer appeared.

In [13]:
from sklearn.utils import class_weight

classes = list(set(answer_tokens))

class_weights = class_weight.compute_class_weight('balanced', classes, answer_tokens)

In [14]:
weights = list(zip(label_encoder.fit_transform(classes), class_weights))
weights.sort(key=lambda x: x[0])

### Model training and checkpointing

In [17]:
filepath="model-checkpoints/answer-embeddings/answer-embeddings-model-{epoch:02d}-{accuracy:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, mode='max')
callbacks_list = [checkpoint]
model.fit(word_embeddings, [y],
                epochs=25,
                batch_size=1000,
                shuffle=True,
                callbacks=callbacks_list)

Epoch 1/25
1038866/1038866 [==============================] - 542s 522us/step - loss: 2.1425 - accuracy: 0.3633

Epoch 00001: saving model to model-checkpoints/answer-embeddings/answer-embeddings-model-01-0.36.hdf5
Epoch 2/25
1038866/1038866 [==============================] - 471s 454us/step - loss: 2.0277 - accuracy: 0.3789

Epoch 00002: saving model to model-checkpoints/answer-embeddings/answer-embeddings-model-02-0.38.hdf5
Epoch 3/25
1038866/1038866 [==============================] - 302s 290us/step - loss: 2.0011 - accuracy: 0.3839

Epoch 00003: saving model to model-checkpoints/answer-embeddings/answer-embeddings-model-03-0.38.hdf5
Epoch 4/25
1038866/1038866 [==============================] - 324s 312us/step - loss: 1.9812 - accuracy: 0.3883

Epoch 00004: saving model to model-checkpoints/answer-embeddings/answer-embeddings-model-04-0.39.hdf5
Epoch 5/25
1038866/1038866 [==============================] - 329s 316us/step - loss: 1.9661 - accuracy: 0.3909

Epoch 00005: saving model t